In [222]:
from sklearnex import patch_sklearn
patch_sklearn()


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [223]:
movies = pd.read_csv('netflix_titles.csv', nrows=1000)
movies.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [224]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1000 non-null   object
 1   type          1000 non-null   object
 2   title         1000 non-null   object
 3   director      685 non-null    object
 4   cast          903 non-null    object
 5   country       694 non-null    object
 6   date_added    1000 non-null   object
 7   release_year  1000 non-null   int64 
 8   rating        1000 non-null   object
 9   duration      1000 non-null   object
 10  listed_in     1000 non-null   object
 11  description   1000 non-null   object
dtypes: int64(1), object(11)
memory usage: 93.9+ KB


In [225]:
movies.isnull().sum()

show_id           0
type              0
title             0
director        315
cast             97
country         306
date_added        0
release_year      0
rating            0
duration          0
listed_in         0
description       0
dtype: int64

In [226]:
# movies.dropna(inplace = True)


In [227]:
from sklearn.preprocessing import LabelEncoder

# # Create the LabelEncoder object
# encoder = LabelEncoder()
# encoder.fit(tracks['artist_name'])
# tracks['artist_name'] = encoder.transform(tracks['artist_name'])

# # Create the LabelEncoder object
# encoder = LabelEncoder()
# encoder.fit(tracks['topic'])
# tracks['topic'] = encoder.transform(tracks['topic'])


# data = movies.drop(['lyrics','track_name'], axis = 1)
# data.head()

In [228]:
movies.drop_duplicates(subset=['title'], keep='first', inplace=True)

In [229]:
movie_vectorizer = CountVectorizer()
movie_vectorizer.fit(movies['type'])

CountVectorizer()

In [230]:

def get_similarities(movie_name, data):
   
  # Getting vector for the input song.
  text_array1 = movie_vectorizer.transform(data[data['title']==movie_name]['type']).toarray()
  num_array1 = data[data['title']==movie_name].select_dtypes(include=np.number).to_numpy()
   
  # We will store similarity for each row of the dataset.
  sim = []
  for idx, row in data.iterrows():
    name = row['title']
     
    # Getting vector for current song.
    text_array2 = movie_vectorizer.transform(data[data['title']==name]['type']).toarray()
    num_array2 = data[data['title']==name].select_dtypes(include=np.number).to_numpy()
 
    # Calculating similarities for text as well as numeric features
    text_sim = cosine_similarity(text_array1, text_array2)[0][0]
    num_sim = cosine_similarity(num_array1, num_array2)[0][0]
    sim.append(text_sim + num_sim)
     
  return sim

In [231]:
def recommend_movies(movie_name, data=movies):
  # Base case
  if movies[movies['title'] == movie_name].shape[0] == 0:
    print('This movie is either not so popular or you\
    have entered invalid_name.\n Some movie you may like:\n')
     
    for mv in data.sample(n=5)['title'].values:
      print(mv)
    return
   
  data['similarity_factor'] = get_similarities(movie_name, data)
 
  data.sort_values(by=['similarity_factor'],
                   ascending = [False],
                   inplace=True)
   
  # First song will be the input song itself as the similarity will be highest.
  display(data[['title', 'type']][:10])

In [233]:
recommend_movies('Naruto Shippûden the Movie: The Will of Fire')

,title,type
594,Star Trek,Movie
450,The Twilight Saga: New Moon,Movie
612,Underworld: Rise of the Lycans,Movie
933,Hachi: A Dog's Tale,Movie
347,Planet 51,Movie
407,Chhota Bheem & Ganesh,Movie
330,Cloudy with a Chance of Meatballs,Movie
947,State of Play,Movie
950,The Lovely Bones,Movie
58,Naruto Shippûden the Movie: The Will of Fire,Movie
